## Salesforce MOIRAI forecaster for sktime

Variations of MOIRAI weights compatible with sktime are present on [sktime's](https://huggingface.co/collections/sktime/moirai-variations-66ba3bc9f1dfeeafaed3b974) hugging face

1. [sktime/moirai-1.0-R-small](https://huggingface.co/sktime/moirai-1.0-R-small)
2. [sktime/moirai-1.0-R-base](https://huggingface.co/sktime/moirai-1.0-R-base)
3. [sktime/moirai-1.0-R-large](https://huggingface.co/sktime/moirai-1.0-R-large)

Note : This code is currently not merged with the main repository. It is a part of the PR [#6746](https://github.com/sktime/sktime/pull/6746).


In [1]:
from sktime.forecasting.moirai_forecaster import MOIRAIForecaster

#intialize the forecaster with wieghts
morai_forecaster = MOIRAIForecaster(checkpoint_path=f"sktime/moirai-1.0-R-small")

/Users/pranavprajapati/miniconda3/envs/moirai-test/lib/python3.9/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [2]:
# Load airline dataset 
from sktime.datasets import load_airline
y = load_airline()
morai_forecaster.fit(y)

MOIRAIForecaster(checkpoint_path='sktime/moirai-1.0-R-small')

In [3]:
# Forecast the next 10 steps
fh = list(range(1, 11))
y_pred = morai_forecaster.predict(fh=fh)

In [4]:
y_pred

Period
1961-01    460.534790
1961-02    479.649414
1961-03    489.711823
1961-04    512.076965
1961-05    503.414764
1961-06    509.496185
1961-07    501.814056
1961-08    502.042816
1961-09    486.840668
1961-10    480.817352
Freq: M, Name: Number of airline passengers, dtype: float32

## Global Forecasting  with MOIRAI without exogenous data

In [4]:
# First we generate 90 series for training and 10 series to predict
from sktime.utils._testing.hierarchical import _make_hierarchical
from sklearn.model_selection import train_test_split

data = _make_hierarchical(
    hierarchy_levels=(100, 1), max_timepoints=10, min_timepoints=10, n_columns=1
)
data = data.droplevel(1)
y_train, y_test = train_test_split(data, test_size=0.1, train_size=0.9, shuffle=False)

y_train is the series we want to fit on which contains the first 90% of the time series instances.

In [5]:
y_train

c0
h0    time                
h0_0  2000-01-01  4.136337
      2000-01-02  5.093438
      2000-01-03  5.633642
      2000-01-04  4.975967
      2000-01-05  2.487956
...                    ...
h0_89 2000-01-06  4.730424
      2000-01-07  4.289083
      2000-01-08  5.480342
      2000-01-09  3.392395
      2000-01-10  3.533849

[900 rows x 1 columns]

Forecast future timepoints based on y_train

In [6]:
morai_forecaster.fit(y_train)
y_pred = morai_forecaster.predict(fh=[1, 2, 3, 4, 5])

In [7]:
# Forecast future timepoints for every instance in y_train
y_pred

c0
h0   time                 
h0_0 2000-01-11  22.784399
     2000-01-12  33.613991
     2000-01-13  20.161270
     2000-01-14  20.910477
     2000-01-15  32.638477
...                    ...
h0_9 2000-01-11  20.871138
     2000-01-12  17.732851
     2000-01-13  12.927707
     2000-01-14  13.084384
     2000-01-15  61.355621

[450 rows x 1 columns]

Now, y_test is the series we want to predict on which contains the last 10% of the time series instances.

In [8]:
y_test

c0
h0    time                
h0_90 2000-01-01  3.919391
      2000-01-02  5.964058
      2000-01-03  3.886492
      2000-01-04  5.520493
      2000-01-05  4.925136
...                    ...
h0_99 2000-01-06  4.407380
      2000-01-07  4.806985
      2000-01-08  3.567953
      2000-01-09  5.988806
      2000-01-10  3.806357

[100 rows x 1 columns]

We can use the already fitted moirai_forecaster to predict on y_test, as MOIRAI is a zero shot model it doesn't perform training in fit and directly does zeorshot forecasting on time series passed in predict.

In [9]:
pred_y_test = morai_forecaster.predict(y=y_test, fh=[1, 2, 3])

This predicts 3 timepoints in the future for every panel in y_test.

In [10]:
pred_y_test

c0
h0    time                 
h0_90 2000-01-11  23.916451
      2000-01-12  15.808086
      2000-01-13  18.153467
h0_91 2000-01-11   8.732608
      2000-01-12  35.566257
      2000-01-13  19.398897
h0_92 2000-01-11  11.376601
      2000-01-12  15.948928
      2000-01-13  21.272480
h0_93 2000-01-11  15.106667
      2000-01-12  16.101036
      2000-01-13  15.571087
h0_94 2000-01-11  25.534584
      2000-01-12  16.738075
      2000-01-13  23.606579
h0_95 2000-01-11  30.656605
      2000-01-12  13.364650
      2000-01-13  22.648291
h0_96 2000-01-11  28.855043
      2000-01-12  45.129093
      2000-01-13  47.197334
h0_97 2000-01-11   5.657803
      2000-01-12   8.085939
      2000-01-13   6.516654
h0_98 2000-01-11   6.295153
      2000-01-12   8.868533
      2000-01-13  19.147938
h0_99 2000-01-11  31.779099
      2000-01-12  28.640955
      2000-01-13   8.717867

### Global Forecasting with MOIRAI with Hierarchical and exogenous data

In [11]:
from sktime.utils._testing.hierarchical import _make_hierarchical
from sklearn.model_selection import train_test_split

data = _make_hierarchical(
    hierarchy_levels=(100, 1), max_timepoints=10, min_timepoints=10, n_columns=2
)
data = data.droplevel(1) # drop the second level of the hierarchy
x = data["c0"].to_frame()
y = data["c1"].to_frame()
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, train_size=0.9, shuffle=False
)
y_test = y_test.groupby(level=0).apply(
    lambda x: x.droplevel(0).iloc[:-3]
)

y_test

In [12]:
y_test

c1
h0    time                
h0_90 2000-01-01  4.867008
      2000-01-02  3.289813
      2000-01-03  6.168579
      2000-01-04  3.507431
      2000-01-05  4.404712
...                    ...
h0_99 2000-01-03  4.799319
      2000-01-04  4.624742
      2000-01-05  3.183761
      2000-01-06  4.086594
      2000-01-07  5.911383

[70 rows x 1 columns]

In [13]:
# initialize the forecaster
morai_forecaster = MOIRAIForecaster(checkpoint_path=f"sktime/moirai-1.0-R-small")
morai_forecaster.fit(y_train, X_train)

MOIRAIForecaster(checkpoint_path='sktime/moirai-1.0-R-small')

In [17]:
y_test

c1
h0    time                
h0_90 2000-01-01  4.867008
      2000-01-02  3.289813
      2000-01-03  6.168579
      2000-01-04  3.507431
      2000-01-05  4.404712
...                    ...
h0_99 2000-01-03  4.799319
      2000-01-04  4.624742
      2000-01-05  3.183761
      2000-01-06  4.086594
      2000-01-07  5.911383

[70 rows x 1 columns]

In [18]:
y_pred = morai_forecaster.predict(fh=[1,2,3], X=X_test, y=y_test)
y_pred

c1
h0    time                
h0_90 2000-01-08  4.255078
      2000-01-09  5.063365
      2000-01-10  5.357237
h0_91 2000-01-08  4.886289
      2000-01-09  5.876314
      2000-01-10  5.894616
h0_92 2000-01-08  6.143803
      2000-01-09  4.628784
      2000-01-10  4.592795
h0_93 2000-01-08  6.056741
      2000-01-09  6.129618
      2000-01-10  6.200014
h0_94 2000-01-08  4.277346
      2000-01-09  4.978453
      2000-01-10  6.043751
h0_95 2000-01-08  5.054675
      2000-01-09  4.287720
      2000-01-10  4.593447
h0_96 2000-01-08  3.822807
      2000-01-09  4.106695
      2000-01-10  3.932107
h0_97 2000-01-08  5.505397
      2000-01-09  5.204144
      2000-01-10  5.055985
h0_98 2000-01-08  4.893248
      2000-01-09  6.020226
      2000-01-10  5.633355
h0_99 2000-01-08  6.307319
      2000-01-09  5.828955
      2000-01-10  5.200515

## Forecasting on Hierarchical time series with MOIRAI

In [21]:
from sktime.utils._testing.hierarchical import _make_hierarchical
from sklearn.model_selection import train_test_split

data = _make_hierarchical(
    hierarchy_levels=(10, 3), max_timepoints=10, min_timepoints=10, n_columns=1
)
data_x = _make_hierarchical(
    hierarchy_levels=(10, 3), max_timepoints=10, min_timepoints=10, n_columns=1
)
#data = data.droplevel(1)
y_train, y_test = train_test_split(data, test_size=0.1, train_size=0.9, shuffle=False)
X_train, X_test = train_test_split(data_x, test_size=0.1, train_size=0.9, shuffle=False)

In [22]:
X_train.rename(columns={'c0': "X"}, inplace=True)
X_test.rename(columns={'c0': "X"}, inplace=True)

Here we try to predict all the time points in `X_test` using `y_train` and `X_train` as the context.

In [23]:
from sktime.forecasting.moirai_forecaster import MOIRAIForecaster
morai_forecaster = MOIRAIForecaster(checkpoint_path=f"sktime/moirai-1.0-R-small", broadcasting=False)
morai_forecaster.fit(y=y_train, X=X_train)
forecast = morai_forecaster.predict(fh=[1,2,3,4,5], X=X_test)

/Users/pranavprajapati/miniconda3/envs/moirai-test/lib/python3.9/site-packages/sktime/forecasting/moirai_forecaster.py:476: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_y.fillna(0, inplace=True)


In [24]:
forecast

c0
0    1    time                
h0_0 h1_0 2000-01-01  0.016382
          2000-01-02  0.036069
          2000-01-03  0.026784
          2000-01-04  0.013065
          2000-01-05  0.022341
...                        ...
h0_9 h1_2 2000-01-01  0.118135
          2000-01-02  0.015807
          2000-01-03  0.023250
          2000-01-04  0.013997
          2000-01-05  0.006157

[150 rows x 1 columns]